# Example: Class with `Streamer` method overridden

## Standalone object

We define a class `TOverrideStreamer` with 2 member variables:

```cpp
class TOverrideStreamer : public TObject {
    int m_int{ 0 };
    double m_double{ 0.0 };

    // ... other codes ...
};
```

Its `Streamer` method is overridden, with additionally writting a "tag" `0x12345678`:

```cpp
void TOverrideStreamer::Streamer( TBuffer& b ) {
    if ( b.IsWriting() )
    {
        TObject::Streamer( b ); // Call base class Streamer
        b << m_int;
        b << 0x12345678; // Write a tag
        b << m_double;
    }
    else
    {
        // ... reading logic ...
    }
}
```

### `Uproot` cannot handle it natively

When reading a `TOverrideStreamer` object with `Uproot`, it will raise an error because `Uproot` does not know how to handle the overridden `Streamer` method.

In [1]:
import uproot

f = uproot.open("demo-data.root")
b = f["my_tree/standalone_obj"]
b.show(name_width=40, typename_width=40)

name                                     | typename                                 | interpretation                
-----------------------------------------+------------------------------------------+-------------------------------
standalone_obj                           | TOverrideStreamer                        | AsStridedObjects(Model_TOverri


In [2]:
try:
    b.array()
except Exception as e:
    print(repr(e))

ValueError('invalid shape in fixed-type tuple: dimension smaller then zero.')


## How `uproot-custom` reads it

![](./uproot-custom-flow.svg)

### C++ "reader" for `TOverrideStreamer`

To read the `TOverrideStreamer` object correctly, we need to implement a custom reader in `uproot-custom` that knows how to skip the tag during reading:

```cpp
class OverrideStreamerReader : public IElementReader {
  private:
    std::shared_ptr<std::vector<int>> m_data_ints;
    std::shared_ptr<std::vector<double>> m_data_doubles;
    // ... other codes ...

  public:
    void read( BinaryBuffer& buffer ) {
        buffer.skip_TObject();

        m_data_ints->push_back( buffer.read<int>() );

        auto tag = buffer.read<uint32_t>();
        if ( tag != 0x12345678 ) throw std::runtime_error( "Error: Unexpected tag value." );

        m_data_doubles->push_back( buffer.read<double>() );
    }

    py::object data() const {
        auto int_array    = make_array( m_data_ints );
        auto double_array = make_array( m_data_doubles );
        return py::make_tuple( int_array, double_array );
    }

    // ... other codes ...
};
```


### Python "factory" for `TOverrideStreamer`

Python factory receives the raw data from the C++ reader and constructs the corresponding `awkward` array and form:

```python
class OverrideStreamerFactory(Factory):
    # ... other codes ...

    def make_awkward_content(self, raw_data):
        int_array, double_array = raw_data

        return awkward.contents.RecordArray(
            [
                awkward.contents.NumpyArray(int_array),
                awkward.contents.NumpyArray(double_array),
            ],
            ["m_int", "m_double"],
        )
```

In [3]:
from uproot_custom import AsCustom, registered_factories

from my_reader import OverrideStreamerFactory

# Register custom factory
registered_factories.add(OverrideStreamerFactory)

# Specify target branches
AsCustom.target_branches.add("/my_tree:standalone_obj")

In [4]:
f = uproot.open("demo-data.root")
b = f["my_tree/standalone_obj"]
b.show(name_width=40, typename_width=40)

name                                     | typename                                 | interpretation                
-----------------------------------------+------------------------------------------+-------------------------------
standalone_obj                           | TOverrideStreamer                        | AsCustom(TOverrideStreamer)   


In [5]:
b.array()

<Array [{m_int: 0, m_double: 0}, ..., {...}] type='10 * {m_int: int32, m_do...'>

## Object in STL

`uproot-custom` can automatically use the above reader and factory to read `TOverrideStreamer` objects stored in STL containers, such as `std::vector<TOverrideStreamer>` and `std::map<int, TOverrideStreamer>`:

In [6]:
AsCustom.target_branches |= {
    "/my_tree:my_obj/m_objVec",
    "/my_tree:my_obj/m_objMap/m_objMap.second",
}

In [7]:
f = uproot.open("demo-data.root")
b = f["my_tree/my_obj"]
b.show(name_width=40, typename_width=40)

name                                     | typename                                 | interpretation                
-----------------------------------------+------------------------------------------+-------------------------------
my_obj                                   | TObjToSave                               | AsGroup(<TBranchElement 'my_ob
TObject                                  | (group of fUniqueID:uint32_t, fBits:u... | AsGroup(<TBranchElement 'TO...
TObject/fUniqueID                        | uint32_t                                 | AsDtype('>u4')
TObject/fBits                            | uint32_t                                 | AsDtype('>u4')
m_objVec                                 | vector<TOverrideStreamer>                | AsCustom(vector<TOverrideSt...
m_objMap                                 | map<int,TOverrideStreamer>               | AsGroup(<TBranchElement 'm_...
m_objMap/m_objMap.first                  | int32_t[]                                | AsJagged(A

In [8]:
b.arrays(filter_name="m_obj*")

<Array [{m_objVec: [...], ...}, ..., {...}] type='10 * {m_objVec: var * {m_...'>

## Using `uproot.dask`

The factory can build the `awkward` form:

```python
class OverrideStreamerFactory(Factory):
    # ... other codes ...
    def make_awkward_form(self):
        return awkward.forms.RecordForm(
            [
                awkward.forms.NumpyForm("int32"),
                awkward.forms.NumpyForm("float64"),
            ],
            ["m_int", "m_double"],
        )
```

wo we can even use `uproot.dask` to read:

In [9]:
arr_dask = uproot.dask("demo-data.root:my_tree/standalone_obj")
arr_dask.type

ArrayType(RecordType([RecordType([NumpyType('int32'), NumpyType('float64')], ['m_int', 'm_double'])], ['standalone_obj']), '??', None)

In [10]:
arr_dask.compute()

<Array [{standalone_obj: {...}}, ..., {...}] type='10 * {standalone_obj: {m...'>